In [1]:
##Recurrent Convolutional Neural Networks (RCNN)
# !conda install -y keras
# !conda install -y tensorflow
# !pip install sklearn

In [24]:
import torch

# a = torch.arange(3*5).reshape(3,5).view(-1)

# b = torch.nonzero(a==10).squeeze()

# print(a,b) # tensor(10).
# print(a[b])

# [0,1,0]   
# [1,0,0]  
# [0,0,1]  

# [5,1,9]   
# [1,60,4]  
# [2,9,1] 

#   x = tf.random_uniform((5, 4))
#   ind = tf.where(x>0.5)
#   y = tf.gather_nd(x, ind)


arr = torch.randint(high=2, size=(3, 3)).reshape(-1)
print(arr)
arr1 = torch.where(arr>0.5)
arr2 = torch.where(arr<0.5)
print(arr1)
# arr = torch.gather(arr,0,arr1[0])
arr = torch.gather(arr,0,arr2[0])
print(arr)

tensor([1, 1, 0, 1, 0, 0, 0, 0, 1])
(tensor([0, 1, 3, 8]),)
tensor([0, 0, 0, 0, 0])


In [ ]:
                answers = answers.view(-1)  #[b,num]
                answer_embedding = answer_embedding.view(-1,size)  #[num，szie]
        
[
    [0,1,0]
    [0,0,1]
]

[
    [6,6,6,6,6,6]
    [6,6,6,6,6,6]
    [6,6,6,6,6,6]
]

In [2]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import GRU
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import LSTM
from keras.datasets import imdb
from keras.datasets import imdb
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os,json

In [3]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=500):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("/ws/code/text-classifi/glove.6B.50d.txt", encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

In [4]:
def Build_Model_RCNN_Text(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50):

    kernel_size = 2
    filters = 256
    pool_size = 2
    gru_node = 256

    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("could not broadcast input array from shape",str(len(embedding_matrix[i])),
                                 "into shape",str(len(embedding_vector))," Please make sure your"
                                 " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)

            embedding_matrix[i] = embedding_vector



    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, recurrent_dropout=0.2))
    model.add(Dense(1024,activation='relu'))
    model.add(Dense(nclasses))
    model.add(Activation('softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [5]:
# newsgroups_train = fetch_20newsgroups(subset='train')
# newsgroups_test = fetch_20newsgroups(subset='test')
# X_train = newsgroups_train.data
# X_test = newsgroups_test.data
# y_train = newsgroups_train.target
# y_test = newsgroups_test.target


train_data = json.load(open("/ws/code/text-classifi/question_relation_train.json", encoding="utf8"))
test_data = json.load(open("/ws/code/text-classifi/question_relation_test.json", encoding="utf8"))
X_train = train_data['data']
X_test = test_data['data']
y_train = train_data['target']
y_test = test_data['target']

# X_test = X_test.to_numpy()
y_train = np.array(y_train)
y_test = np.array(y_test)


X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test)

Found 2985 unique tokens.
(5492, 500)
Total 400000 word vectors.


In [6]:
model_RCNN = Build_Model_RCNN_Text(word_index,embeddings_index, 20)


model_RCNN.summary()

model_RCNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=15,
                              batch_size=128,
                              verbose=2)

predicted = model_RCNN.predict(X_test_Glove)

predicted = np.argmax(predicted, axis=1)
print(metrics.classification_report(y_test, predicted))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 50)           149300    
_________________________________________________________________
dropout (Dropout)            (None, 500, 50)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 499, 256)          25856     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 249, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 248, 256)          131328    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 124, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 123, 256)          1

InvalidArgumentError:  Received a label value of 48 which is outside the valid range of [0, 20).  Label values: 2 7 2 2 0 9 3 4 5 4 4 1 4 11 6 0 0 4 26 5 1 2 17 0 3 4 3 3 8 1 2 1 7 2 3 0 0 1 4 4 6 3 17 0 3 3 3 5 3 0 2 1 4 1 3 2 0 9 2 5 6 5 4 1 2 2 4 3 1 1 7 13 3 6 5 1 5 0 48 5 1 5 1 29 4 0 13 0 1 3 8 3 2 3 1 2 6 3 7 0 3 2 4 5 7 2 5 3 5 0 3 3 1 5 3 0 4 4 3 5 2 2 0 3 1 6 4 2
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-6-bab515a22f93>:10) ]] [Op:__inference_train_function_12760]

Function call stack:
train_function
